In [19]:
from __future__ import division
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
from sklearn.decomposition import PCA
import pandas as pd
import math
from numpy import random
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize as norm
from sklearn.metrics import roc_curve, roc_auc_score
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import f1_score

In [12]:
df = pd.read_excel('../data/CTG.xls', sheet_name='Raw Data', header=0, skiprows=[1])
data = df.to_numpy()
# Features matrix
X = data[:,0:22]
Y = data[:,23]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0, stratify=Y, shuffle=True)

In [14]:
smt = SMOTE()
X_smt, Y_smt = smt.fit_resample(X_train, y_train)
X_smt.shape

(3723, 22)

In [3]:
def prueba_svm(ker='linear',conf=0.001,gam=0.0):
    Folds = 4
    random.seed(19680801)
    EficienciaTrain = np.zeros(Folds)
    EficienciaVal = np.zeros(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    porc_vectores = np.zeros(Folds)
    j = 0
    
    for train, test in skf.split(X, Y):
        Xtrain = X[train,:]
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]

        #Normalizamos los datos
        scaler = preprocessing.StandardScaler().fit(Xtrain)
        Xtrain = scaler.transform(Xtrain)
        Xtest = scaler.transform(Xtest)
        
        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        if ker == 'linear':
            modelo = SVC(kernel =ker, C=conf)
        else:
            modelo = SVC(kernel=ker, C= conf, gamma= gam)
        modelo.fit(Xtrain,Ytrain)


        #Validación
        Ytrain_pred = modelo.predict(Xtrain)
        Yest = modelo.predict(Xtest)

        #Evaluamos las predicciones del modelo con los datos de test
        EficienciaTrain[j] = np.mean(Ytrain_pred.ravel() == Ytrain.ravel())
        EficienciaVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
        porc_vectores[j] = len(modelo.support_vectors_)/len(Xtrain)
        j += 1

    print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
    print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
    
    eficiencia_val = np.mean(EficienciaVal)
    ic_val = np.std(EficienciaVal)
    eficiencia_train = np.mean(EficienciaTrain)
    ic_train = np.std(EficienciaTrain)
    porc_vect = np.mean(porc_vectores)
    
    return eficiencia_val,ic_val,eficiencia_train,ic_train,porc_vect

In [1]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'Kernel' : pd.Series(['linear','linear','linear','linear','linear','linear','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]),
    'gamma' : pd.Series([0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1])})
df_types["Eficiencia en validacion"] = ""
df_types["Intervalo de confianza"] = ""
df_types["Eficiencia en entrenamiento"] = ""
df_types["Intervalo de confianza"] = ""
df_types["% de Vectores de Soporte"] = ""
df_types.set_index(['Kernel','C','gamma'], inplace=True)

In [6]:
j = 0
for i in df_types.index:
    print(i)
    kernel = i[0]
    c = i[1]
    gamma = i[2]    
    eficiencia_val,ic_val,eficiencia_train,ic_train,porc_vect=prueba_svm(ker=kernel,conf=c,gam=gamma)
    df_types.loc[i,"Eficiencia en validacion"] = str(eficiencia_val)
    df_types.loc[i,"Intervalo de confianza"] = str(ic_val)
    df_types.loc[i,"Eficiencia en entrenamiento"] = str(eficiencia_train)
    df_types.loc[i,"Intervalo de confianza"] = str(ic_train)
    df_types.loc[i, "% de Vectores de Soporte"] = str(porc_vect)
    j = j+1
    print(eficiencia_val,ic_val,eficiencia_train,ic_train,porc_vect)

('linear', 0.001, 0.0)
Eficiencia durante el entrenamiento = 0.8162457766782173+-0.008395656079409227
Eficiencia durante la validación = 0.8137566727553347+-0.02615462018078472
0.8137566727553347 0.02615462018078472 0.8162457766782173 0.008395656079409227 0.4137694056473531
('linear', 0.01, 0.0)
Eficiencia durante el entrenamiento = 0.8958917256325641+-0.012133197174505657
Eficiencia durante la validación = 0.8349032892966881+-0.02246488742738512
0.8349032892966881 0.02246488742738512 0.8958917256325641 0.012133197174505657 0.3264359294061193
('linear', 0.1, 0.0)
Eficiencia durante el entrenamiento = 0.9167451021267056+-0.007922726755760525
Eficiencia durante la validación = 0.8372440635487022+-0.04777071892472698
0.8372440635487022 0.04777071892472698 0.9167451021267056 0.007922726755760525 0.2568217217386516
('linear', 1.0, 0.0)
Eficiencia durante el entrenamiento = 0.9244270245395154+-0.00584971906888737
Eficiencia durante la validación = 0.8391149129886865+-0.055567550581846384
0.8

In [20]:
def train_svm(kernel, C, gamma):
    Folds = 4
    random.seed(19680801)
    f1 = np.zeros(Folds)
    auc_fpr = np.zeros(Folds)
    gmean = np.zeros(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    models = []
    for train, test in skf.split(X_smt, Y_smt):
        Xtrain = X_smt[train,:]
        Ytrain = Y_smt[train]
        Xtest = X_smt[test,:]
        Ytest = Y_smt[test]

        #Normalizamos los datos
        Xtrain = norm(Xtrain)
        Xtest = norm(Xtest)

        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        model = SVC(C=C,kernel=kernel, gamma=gamma, probability=True)
        model.fit(Xtrain, Ytrain)
        models.append(model)
        #Validación
        Y_pred=model.predict_proba(Xtest)
        Ytrain_pred = model.predict(Xtrain)#Use el modelo previamente entrenado para hacer predicciones con las mismas muestras de entrenamiento
        Yest = model.predict(Xtest)#Use el modelo previamente entrenado para hacer predicciones con las muestras de test

        #Evaluamos las predicciones del modelo con los datos de test

        auc_fpr[j] = roc_auc_score(Ytest, Y_pred, multi_class="ovr", average="weighted")
        gmean[j] =  geometric_mean_score(y_true = Ytest, y_pred=Yest, average="weighted")
        f1[j] = f1_score(y_true = Ytest, y_pred=Yest, average = "weighted")
        
        j += 1
    print(f"Finalizado: {kernel} C: {C} gamma: {gamma}")
    return str(np.mean(f1)), str(np.std(f1)), str(np.mean(auc_fpr)), str(np.std(auc_fpr)), str(np.mean(gmean)), str(np.std(gmean))

In [21]:
import qgrid
randn = np.random.randn
df_types = pd.DataFrame({
    'Kernel' : pd.Series(['lineal','lineal','lineal','lineal','lineal','lineal','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]),
    'gamma' : pd.Series([0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1])})

f1_ = []
stdf1_ = []
auc_ =[]
stdAuc_ = []
gmean_ = []
stdgmean_ = []
for e, i, j in zip(df_types['Kernel'], df_types['C'], df_types['gamma']):
    if(e == 'lineal'):
        f1, stdf1, auc, stdAuc, gmean, stdgmean = train_svm("linear", i, "auto")
    else:
        f1, stdf1, auc, stdAuc, gmean, stdgmean = train_svm(e, i, j)
    f1_.append(f1)
    stdf1_.append(stdf1)
    auc_.append(auc)
    stdAuc_.append(stdAuc)
    gmean_.append(gmean)
    stdgmean_.append(stdgmean)
df_types.set_index(['Kernel','C','gamma'], inplace=True)
df_types["f1"] = f1_
df_types["Intervalo de confianza f1"] = stdf1_
df_types["AUC"] = auc_
df_types["Intervalo de confianza AUC"] = stdAuc_
df_types["G-mean"] = gmean_
df_types["Intervalo de confianza G-mean"] = stdgmean_

Finalizado: linear C: 0.001 gamma: auto
Finalizado: linear C: 0.01 gamma: auto
Finalizado: linear C: 0.1 gamma: auto
Finalizado: linear C: 1.0 gamma: auto
Finalizado: linear C: 10.0 gamma: auto
Finalizado: linear C: 100.0 gamma: auto
Finalizado: rbf C: 0.001 gamma: 0.01
Finalizado: rbf C: 0.001 gamma: 0.1
Finalizado: rbf C: 0.001 gamma: 1.0
Finalizado: rbf C: 0.01 gamma: 0.01
Finalizado: rbf C: 0.01 gamma: 0.1
Finalizado: rbf C: 0.01 gamma: 1.0
Finalizado: rbf C: 0.1 gamma: 0.01
Finalizado: rbf C: 0.1 gamma: 0.1
Finalizado: rbf C: 0.1 gamma: 1.0
Finalizado: rbf C: 1.0 gamma: 0.01
Finalizado: rbf C: 1.0 gamma: 0.1
Finalizado: rbf C: 1.0 gamma: 1.0
Finalizado: rbf C: 10.0 gamma: 0.01
Finalizado: rbf C: 10.0 gamma: 0.1
Finalizado: rbf C: 10.0 gamma: 1.0
Finalizado: rbf C: 100.0 gamma: 0.01
Finalizado: rbf C: 100.0 gamma: 0.1
Finalizado: rbf C: 100.0 gamma: 1.0


In [22]:
df_types

f1 Intervalo de confianza f1  \
Kernel C       gamma                                                  
lineal 0.001   0.00     0.400985396625114       0.08114168164757067   
       0.010   0.00     0.400985396625114       0.08114168164757067   
       0.100   0.00    0.6773160378086234      0.008269495900968716   
       1.000   0.00    0.7944521345530227      0.012604506130272107   
       10.000  0.00    0.7998211136118669       0.01034442475794446   
       100.000 0.00    0.8488015729844497     0.0040557027209568546   
rbf    0.001   0.01    0.4007981013710985       0.08126035567151259   
               0.10    0.4012914412554785       0.08177643862294436   
               1.00   0.41916149746964493       0.08558116814158921   
       0.010   0.01    0.4007981013710985       0.08126035567151259   
               0.10    0.4012914412554785       0.08177643862294436   
               1.00    0.4571328546717835       0.06333213730473149   
       0.100   0.01    0.4007981013710985       0.08126035567151259   
               0.10    0.4770959937863151       0.04080286724630027   
               1.00    0.7041154259887312      0.005087532207061886   
       1.000   0.01   0.47845843251935527      0.040397576764320584   
               0.10    0.7079434118121507      0.004846635007725406   
               1.00    0.7968500099907807      0.008173035456999597   
       10.000  0.01    0.7082429878128415       0.00521795240539875   
               0.10     0.792546530321085      0.010306644249311915   
               1.00    0.8372598014936714      0.006234213216458686   
       100.000 0.01    0.7913051228841417      0.008564279182970326   
               0.10    0.8279715125795155       0.00913353723512611   
               1.00    0.8771960304390385      0.006674933641395463   

                                      AUC Intervalo de confianza AUC  \
Kernel C       gamma                                                   
lineal 0.001   0.00   0.30730689196275934        0.11325295381575672   
       0.010   0.00    0.7538385568160542        0.00632819847530656   
       0.100   0.00    0.8694685631853851        0.00715740707095775   
       1.000   0.00    0.9313996986803229       0.005659200351229949   
       10.000  0.00    0.9427290667271839       0.004916536292226425   
       100.000 0.00    0.9580909984928973      0.0028561495749498725   
rbf    0.001   0.01   0.22083419531554782       0.007781533064275687   
               0.10   0.21886415100544676       0.008405450418679917   
               1.00   0.28313664017397844        0.12725643746729132   
       0.010   0.01   0.22079649590011025       0.008453601274172462   
               0.10    0.2801873764728614        0.12973004363616092   
               1.00    0.7966584070679367       0.008885927760979736   
       0.100   0.01   0.27947738085334445        0.13025536214418101   
               0.10    0.7926559388586649       0.003814857449698414   
               1.00    0.9020326718301979       0.006357776106774127   
       1.000   0.01    0.7920969488618468      0.0035287675054353914   
               0.10    0.8992816839897069       0.007569575142645369   
               1.00    0.9360911826364845       0.004762246599295739   
       10.000  0.01     0.898742376757048       0.007767695855737559   
               0.10    0.9354797238707326      0.0052443576169826575   
               1.00     0.951735574008962      0.0038490849164493587   
       100.000 0.01    0.9352668980983262       0.005353942529951851   
               0.10    0.9481118094004877      0.0042076099995313285   
               1.00    0.9722577529223014      0.0015752559473097723   

                                  G-mean Intervalo de confianza G-mean  
Kernel C       gamma                                                    
lineal 0.001   0.00   0.5946569636502275           0.04185279077087604  
       0.010   0.00   0.5946569636502275           0.04185279077087604  
       0.100   0.00    0.74